In [1]:
import re
import sys
import glob
import math
import optuna
import numpy as np
import pandas as pd
import seaborn as sns
from time import ctime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import lightgbm as lgb
from sklearn.model_selection import KFold
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_frags = glob.glob("./train_fft/*")
test_frags = glob.glob("./test_fft/*")
train = pd.read_csv("train.csv")
Y = pd.Series(0, index=np.arange(len(train_frags)))

i = 0
for file in train_frags:
    start = './train_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]
    t2e = int(train.loc[train['segment_id'] == int(seg_id)]['time_to_eruption'].values)
    Y.iloc[i] = t2e
    i = i + 1

test_id = pd.Series(0, index=np.arange(len(test_frags)))    
i = 0
for file in test_frags:
    start = './test_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]

    test_id.iloc[i] = seg_id
    i = i + 1


In [3]:
def objective(trial):
    params = { "objective": "regression",
              "metric": "mae",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "num_iterations": 1000,
              "early_stopping_round": 5,
              "n_jobs": -1,
              'num_leaves': trial.suggest_int("nleaves", 32, 256),
              'bagging_fraction': trial.suggest_float('bagfrac',0.7,1.0),
              'bagging_freq': trial.suggest_int("bagfreq", 1, 10),
              'feature_fraction': trial.suggest_float('featfrac',0.7,1.0),
              'lambda_l1': trial.suggest_float("lambda_l1", 1e-6, 10.0, log=True),
              'lambda_l2': trial.suggest_float("lambda_l2", 1e-6, 10.0, log=True),
              'min_child_samples': trial.suggest_int("minchild", 1, 100)}
    
    all_pca = pd.read_csv('pca_30.csv')

    X = all_pca[:len(train_frags)]
    X_test = all_pca[-len(test_frags):]

    n_fold = 5
    mae, r2 = [], []
    predicted_times = np.zeros(len(X_test))
    cv = KFold(n_splits=n_fold, shuffle=True, random_state=42)


    for fold_n, (train_index, valid_index) in enumerate(cv.split(X)):

        if fold_n == 4:
            X_train = X.iloc[train_index,:]
            X_valid = X.iloc[valid_index,:]

            Y_train = Y.iloc[train_index]
            Y_valid = Y.iloc[valid_index]

            dtrain = lgb.Dataset(X_train, label=Y_train)
            dval = lgb.Dataset(X_valid, label=Y_valid)

            model = lgb.train( params, dtrain, valid_sets=[dtrain, dval], verbose_eval=-1 )

            y_pred = model.predict(X_valid)
            mae.append(mean_absolute_error(Y_valid, y_pred))
            r2.append(r2_score(Y_valid, y_pred))

            predicted_times += model.predict(X_test)

#     predicted_times /= n_fold

    filename = 'fold_submission' + str(trial.number) + '.csv'
    submission = pd.DataFrame({
            "segment_id": test_id,
            "time_to_eruption": predicted_times
        })

    submission.to_csv(filename, index=False)
    
    return np.mean(mae)



In [4]:
study_name = 'lgbm_study' 
study = optuna.create_study(direction="minimize",load_if_exists = True,study_name=study_name, storage='sqlite:///lgbm_study.db')
study.optimize(objective, n_trials=1000)

[I 2021-01-03 17:55:47,274] A new study created in RDB with name: lgbm_study


Training until validation scores don't improve for 5 rounds


[I 2021-01-03 17:55:48,350] Trial 0 finished with value: 3713619.1573620606 and parameters: {'nleaves': 254, 'bagfrac': 0.9699256013710338, 'bagfreq': 6, 'featfrac': 0.9646401832292606, 'lambda_l1': 0.0003197445256710398, 'lambda_l2': 6.97323177646941e-05, 'minchild': 55}. Best is trial 0 with value: 3713619.1573620606.


Early stopping, best iteration is:
[207]	training's l1: 818189	valid_1's l1: 3.71362e+06
Training until validation scores don't improve for 5 rounds


[I 2021-01-03 17:55:49,314] Trial 1 finished with value: 3719101.063892645 and parameters: {'nleaves': 215, 'bagfrac': 0.7284073359868097, 'bagfreq': 5, 'featfrac': 0.8858721259678166, 'lambda_l1': 5.919475516936757e-06, 'lambda_l2': 0.003127738873385397, 'minchild': 32}. Best is trial 0 with value: 3713619.1573620606.


...

Early stopping, best iteration is:
[198]	training's l1: 316139	valid_1's l1: 3.70277e+06
Training until validation scores don't improve for 5 rounds


[I 2021-01-03 18:23:23,226] Trial 998 finished with value: 3711111.9731306555 and parameters: {'nleaves': 45, 'bagfrac': 0.9234974531159372, 'bagfreq': 1, 'featfrac': 0.8908949175885941, 'lambda_l1': 7.10930957771808e-06, 'lambda_l2': 3.4657688078672937, 'minchild': 4}. Best is trial 496 with value: 3511896.0883319825.


Early stopping, best iteration is:
[214]	training's l1: 576845	valid_1's l1: 3.71111e+06
Training until validation scores don't improve for 5 rounds


[I 2021-01-03 18:23:25,027] Trial 999 finished with value: 3708991.883156082 and parameters: {'nleaves': 63, 'bagfrac': 0.9413426425284047, 'bagfreq': 2, 'featfrac': 0.8827255594560228, 'lambda_l1': 0.00028112686801626144, 'lambda_l2': 7.288936891502989, 'minchild': 1}. Best is trial 496 with value: 3511896.0883319825.


Early stopping, best iteration is:
[169]	training's l1: 534229	valid_1's l1: 3.70899e+06


In [5]:
fold0 = pd.read_csv('fold_0.csv')
fold1 = pd.read_csv('fold_1.csv')
fold2 = pd.read_csv('fold_2.csv')
fold3 = pd.read_csv('fold_3.csv')
fold4 = pd.read_csv('fold_4.csv')

In [13]:
fold0.columns

Index(['segment_id', 'time_to_eruption'], dtype='object')

In [12]:
time_to_eruption = fold0.time_to_eruption + fold1.time_to_eruption + fold2.time_to_eruption + fold3.time_to_eruption + fold4.time_to_eruption
time_to_eruption /= 5

In [14]:
submission = pd.DataFrame({
        "segment_id": fold0.segment_id,
        "time_to_eruption": time_to_eruption
    })

submission.to_csv('fold_composite.csv', index=False)